In [47]:
from os import listdir
from os.path import isfile, join 
import numpy as np
import pandas as pd 
import datetime as dt

path="C:/Users/Luis Moros/Documents/GitHub/MAN6915/FinalProject"

In [48]:
def obtainRawFileNames(path,ext):
    extension = ext if "." in ext else ".{}".format(ext)
    return ["{}/{}".format(path,f) 
            for f in listdir(path)
            if isfile(join(path, f)) 
            and f.endswith(extension)]

In [49]:
def normalizedFields(line, separator):
    fields = [field.replace('|', ' ') for field in line.split(',')]
    if len(fields)<4:
        return ""
    return separator.join((fields[0], " ".join(fields[1:-2]), fields[-2], fields[-1]))

In [50]:
def filterEmptyLines(lines):
    return (line for line in lines if line != "")

In [51]:
def replaceComasByPipes(path):
    with open(path, 'r', encoding="Latin-1") as file :
        filedata = file.read()
    
    filelines = filterEmptyLines(
                    (normalizedFields(line, '|') 
                     for line in filedata.split('\n')))

    # Write the file out again
    with open(path.replace('.','_cleaned.'), 'w', encoding="Latin-1") as file:
        file.write("\n".join(filelines))

In [52]:
print("\n".join(obtainRawFileNames(
                path,
                "csv"
        )))

C:/Users/Luis Moros/Documents/GitHub/MAN6915/FinalProject/assigned_to.csv
C:/Users/Luis Moros/Documents/GitHub/MAN6915/FinalProject/assigned_to_cleaned.csv
C:/Users/Luis Moros/Documents/GitHub/MAN6915/FinalProject/assigned_to_cleaned_cleaned.csv
C:/Users/Luis Moros/Documents/GitHub/MAN6915/FinalProject/assigned_to_cleaned_cleaned_cleaned.csv
C:/Users/Luis Moros/Documents/GitHub/MAN6915/FinalProject/bug_status.csv
C:/Users/Luis Moros/Documents/GitHub/MAN6915/FinalProject/bug_status_cleaned.csv
C:/Users/Luis Moros/Documents/GitHub/MAN6915/FinalProject/bug_status_cleaned_cleaned.csv
C:/Users/Luis Moros/Documents/GitHub/MAN6915/FinalProject/bug_status_cleaned_cleaned_cleaned.csv
C:/Users/Luis Moros/Documents/GitHub/MAN6915/FinalProject/cc.csv
C:/Users/Luis Moros/Documents/GitHub/MAN6915/FinalProject/cc_cleaned.csv
C:/Users/Luis Moros/Documents/GitHub/MAN6915/FinalProject/cc_cleaned_cleaned.csv
C:/Users/Luis Moros/Documents/GitHub/MAN6915/FinalProject/cc_cleaned_cleaned_cleaned.csv
C:/Users

In [53]:
#for file in obtainRawFileNames(path, "csv"):
#    replaceComasByPipes(file)       

In [54]:
durationDf = pd.read_csv("{}/{}".format(path,"resolution_cleaned.csv"),sep='|')

In [55]:
def calculateDuration(x):
    if len(x) ==1:
        arbitraryMax = ((dt.datetime(2013,1,1) - dt.datetime(1970,1,1)).total_seconds())
        return arbitraryMax - min(x) 
    return max(x) - min(x)

durationDf["when"] = durationDf["when"].apply(pd.to_numeric)
durationDfFinal = pd.DataFrame(durationDf.groupby("id")["when"].apply(calculateDuration))
durationDfFinal.columns = ['duration']

In [56]:
ccDf = pd.read_csv("{}/{}".format(path,"cc_cleaned.csv"),sep='|', na_filter=False)

In [57]:
def calculateTotalCcs(x):
    ccs = x.str.split(';').count()
    return ccs

ccDfFinal = ccDf[ccDf["what"].str.strip()!='']
ccDfFinal = pd.DataFrame(ccDfFinal.groupby("id")["what"].apply(calculateTotalCcs))
ccDfFinal.columns = ['totalCcs']

In [58]:
openMonthDf = pd.read_csv("{}/{}".format(path,"reports_cleaned.csv"),sep='|')

In [59]:
def calculateOpenMonth(column):
    dates = pd.to_datetime(column,unit='s')
    return dates.map(lambda x : x.month)
    
openMonthDf["opening"] = openMonthDf["opening"].apply(pd.to_numeric)
openMonthDfFinal = pd.DataFrame(openMonthDf.groupby("id")["opening"].apply(calculateOpenMonth))
openMonthDfFinal.columns=['openMonth']

In [60]:
severityDf = pd.read_csv("{}/{}".format(path,"severity_cleaned.csv"),sep='|')

In [61]:
severityDfFinal = pd.DataFrame(severityDf.groupby('id').first()['what'])
severityDfFinal.columns = ['severity']

In [62]:
eventDf = pd.read_csv("{}/{}".format(path,"resolution_cleaned.csv"),sep='|')

In [63]:
eventDf.sort_values("id")
eventDfFinal = pd.DataFrame(eventDf.groupby('id').last()['what']\
                            .apply(lambda x: 1 if x=='FIXED' else 0))
eventDfFinal.columns = ['event']

In [64]:
finalDF = durationDfFinal\
            .join(ccDfFinal, how='left')\
            .join(openMonthDfFinal, how='inner' )\
            .join(severityDfFinal, how='inner')\
            .join(eventDfFinal, how='inner')
finalDF.fillna(value=0, inplace=True) 
finalDF

,duration,totalCcs,openMonth,severity,event
1,759674.0,8.0,8,normal,1
2,17434559.0,3.0,12,normal,1
3,18125013.0,0.0,1,normal,1
4,12253962.0,0.0,5,normal,1
5,136556.0,0.0,7,normal,0
6,10353286.0,0.0,8,normal,0
7,112650241.0,1.0,3,normal,0
8,137820.0,0.0,3,normal,0
9,137850.0,0.0,3,normal,0
10,138040.0,1.0,3,normal,1
